In [1]:
import pandas as pd
import numpy as np
import calendar
import seaborn as sns
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTENC
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# pip install git+https://github.com/scikit-learn-contrib/category_encoders.git#egg=httpie
# pip install imblearn

Using TensorFlow backend.


## **刪除沒有被保人資料的列**

In [4]:
df = pd.read_excel('理賠再購屬性合併1_2.xlsx',index_col=[0,1])

In [12]:
df = df.dropna(subset=['CUST_RK'])

In [13]:
df.to_excel('理賠再購屬性合併1-2_before_encoding.xlsx')

## **匯入資料**

In [2]:
df = pd.read_excel('理賠再購屬性合併1-2_before_encoding.xlsx',index_col=[0,1])

## **特徵工程**

In [4]:
df = df[['再購(120天)',
       '再購(180天)', '再購(360天)','INJURED_RK', 'Claim_RK', 'BundleSubtype2', 'illness_code',
       'illness_desc', 'DiagnosisCode_DESC', 'claim_settle_dt',
       'REIMBURSED_YR_TW', 'POLICY_HOLDER_RK', 'MATURITY_BENEFICIARY_RK',
       'DEATH_BENEFICIARY_RK', '結案後120天', '結案後180天', '結案後360天', 'CUST_RK', 'ternure_m', 'recency_m', 'SIN',
       'SIN_his', 'REG', 'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb',
       'AHb_his', 'AHc', 'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP',
       'WEALTH_LEVEL', 'CLIENT_MARITAL', 'CLIENT_INCOME', 'DIGI_FLG',
       'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2', 'TOTAL_AUM',
       'INSURED_DOB']]

In [6]:
X = df.iloc[:,3:]
y = df.iloc[:,0]

In [8]:
X = X.drop(columns=['INJURED_RK', 'Claim_RK','illness_code','claim_settle_dt','POLICY_HOLDER_RK','MATURITY_BENEFICIARY_RK',
       'DEATH_BENEFICIARY_RK', '結案後120天', '結案後180天', '結案後360天','CUST_RK','CLIENT_MARITAL','CLIENT_INCOME','TOTAL_AUM','INSURED_DOB'])

In [9]:
X.tail()

,,BundleSubtype2,illness_desc,DiagnosisCode_DESC,REIMBURSED_YR_TW,ternure_m,recency_m,SIN,SIN_his,REG,REG_his,...,AHd,AHd_his,VIP_CLASS,VIP,WEALTH_LEVEL,DIGI_FLG,TOPCARD,GENDER,stick_level2,cust_group2
INSURED_RK,Policy_RK,,,,,,,,,,,,,,,,,,,,,
251796,74433,6.N意外醫療,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,4389.00,321,201,0,0,1,1,...,0,0,NaN,0,W6,1,0,1,S09,G4
251813,78696,6.N意外醫療,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,31133.55,263,240,0,0,0,0,...,0,0,NaN,0,W6,0,0,1,S09,G4
251876,57631,6.N意外醫療,涉及其他特定運動和競技之活動,20.疾病和死亡的外因,1470.00,285,13,0,0,1,1,...,0,0,NaN,0,W5,0,0,1,S03,G2
251904,105696,5.N疾病醫療,其他及未明示部位之原位癌,02.腫瘤,17002.65,317,282,0,0,1,1,...,0,0,NaN,0,W6,0,0,0,S10,G4
251954,74765,4.C重大疾病,自體的冠狀動脈粥樣硬化心臟病,09.循環系統疾病,16825.20,297,68,0,0,1,1,...,0,0,NaN,0,W6,0,0,1,S07,G4


In [10]:
X.columns

Index(['BundleSubtype2', 'illness_desc', 'DiagnosisCode_DESC',
       'REIMBURSED_YR_TW', 'ternure_m', 'recency_m', 'SIN', 'SIN_his', 'REG',
       'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb', 'AHb_his', 'AHc',
       'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP', 'WEALTH_LEVEL',
       'DIGI_FLG', 'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2'],
      dtype='object')

In [11]:
X['VIP_CLASS'] = X['VIP_CLASS'].fillna('0')

In [12]:
X.reset_index(drop=True, inplace=True)

In [21]:
X.tail()

,BundleSubtype2,illness_desc,DiagnosisCode_DESC,REIMBURSED_YR_TW,ternure_m,recency_m,SIN,SIN_his,REG,REG_his,...,AHd,AHd_his,VIP_CLASS,VIP,WEALTH_LEVEL,DIGI_FLG,TOPCARD,GENDER,stick_level2,cust_group2
220967,6.N意外醫療,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,4389.00,321,201,0,0,1,1,...,0,0,0,0,W6,1,0,1,S09,G4
220968,6.N意外醫療,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,31133.55,263,240,0,0,0,0,...,0,0,0,0,W6,0,0,1,S09,G4
220969,6.N意外醫療,涉及其他特定運動和競技之活動,20.疾病和死亡的外因,1470.00,285,13,0,0,1,1,...,0,0,0,0,W5,0,0,1,S03,G2
220970,5.N疾病醫療,其他及未明示部位之原位癌,02.腫瘤,17002.65,317,282,0,0,1,1,...,0,0,0,0,W6,0,0,0,S10,G4
220971,4.C重大疾病,自體的冠狀動脈粥樣硬化心臟病,09.循環系統疾病,16825.20,297,68,0,0,1,1,...,0,0,0,0,W6,0,0,1,S07,G4


In [22]:
X.columns

Index(['BundleSubtype2', 'illness_desc', 'DiagnosisCode_DESC',
       'REIMBURSED_YR_TW', 'ternure_m', 'recency_m', 'SIN', 'SIN_his', 'REG',
       'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb', 'AHb_his', 'AHc',
       'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP', 'WEALTH_LEVEL',
       'DIGI_FLG', 'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2'],
      dtype='object')

## **處理資料不平衡**
[不平衡資料的二元分類 2：利用抽樣改善模型品質](https://taweihuang.hpd.io/2018/12/30/imbalanced-data-sampling-techniques/) <br>
[Oversampling: SMOTE for binary and categorical data in Python](https://stackoverflow.com/questions/47655813/oversampling-smote-for-binary-and-categorical-data-in-python)

In [31]:
cate = [0,1,2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] # 這些是類別資料，使用smotenc前要先標出來
sm = SMOTENC(categorical_features = cate, random_state=0)
X_res, y_res = sm.fit_resample(X, y)
df1 = X_res
df1['y'] = y_res

## **encoding**

In [32]:
## 要先分訓練跟測試，才能target encoding
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.33, random_state=42)
to_encoding = ['BundleSubtype2', 'illness_desc', 'DiagnosisCode_DESC','VIP_CLASS','WEALTH_LEVEL','stick_level2', 'cust_group2']
enc = TargetEncoder(cols=to_encoding)
training_numeric_dataset = enc.fit_transform(X_train, y_train)
testing_numeric_dataset = enc.transform(X_test)

In [36]:
training_numeric_dataset.shape

(280896, 29)

## **儲存最後training, testing data**

In [34]:
training_numeric_dataset.to_excel('training_data_baseline.xlsx')
testing_numeric_dataset.to_excel('testing_data_baseline.xlsx')